<a href="https://colab.research.google.com/github/roitraining/SparkProgram/blob/Day3/Day3/Ch04_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Open a terminal window and run the following commands:
sudo bash
start-hadoop
cd /home/student/ROI/SparkProgram/Day3
./fixhive.sh


## Let's make a simple hive table for regions.

In [0]:
! cat /home/student/ROI/SparkProgram/Day3/regions.hql

In [0]:
import sys
sys.path.append('/home/student/ROI/SparkProgram')
from initspark import *
sc, spark, conf = initspark()

## You can query an existing Hive table and bring it into a Spark DataFrame.

In [0]:
regions = spark.sql('select * from regions')
regions.show()
print(regions)

## Read in a file to a Spark DataFrame.

In [0]:
territories = spark.read.csv('/home/student/ROI/SparkProgram/datasets/northwind/CSVHeaders/territories', header=True)
territories.show()

## Use createOrReplaceTempView to create a virtual table in the Hive catalog and then it can be queried using SQL as if it were a hive table.

In [0]:
territories.createOrReplaceTempView('territories')
t1 =spark.sql('select * from territories where regionid = 1')
t1.show()
print(t1.count())

## Spark DataFrames can be saved to a Hive table using either the saveAsTable method or writing a SQL query that uses CREATE TABLE AS.

In [0]:
territories.write.saveAsTable('Territories2', mode='overwrite')
spark.sql('create table Territories3 as select * from territories')

## Queries use standard HQL to mix Hive tables and virtual tables. Both are read into a Spark DataFrame and the processing happens at the Spark level, not at the Hive level. HQL is just used to parse the logic into the corresponding Spark methods.

In [0]:
sql = """select r.regionid, r.regionname, t.territoryid, t.territoryname 
from regions as r 
join territories as t on r.regionid = t.regionid 
order by r.regionid, t.territoryid"""
rt = spark.sql(sql)
rt.show(10)

tr = regions.join(territories, regions.regionid == territories.RegionID). \
     select('regions.regionid', 'regionname', 'TerritoryID', 'TerritoryName')
tr.show(10)

## Lab: Read the northwind JSON products and make it into a TempView and do the same with the CSVHeaders version of categories.

## Install the MySQL Python connector. This has nothing to do with Spark, but if you want to run SQL queries directly it is helpful.

In [0]:
! pip install mysql-connector-python

## Let's make sure we have a database for northwind and no regions table.

In [0]:
import mysql.connector
try:
    cn = mysql.connector.connect(host='localhost', user='test', password='password')
    cursor = cn.cursor()
    cursor.execute('create database if not exists northwind')
    cn.close()

    cn = mysql.connector.connect(host='localhost', user='test', password='password', database='northwind')
    cursor = cn.cursor()    
    cursor.execute('drop table if exists regions')
    cn.close()
except:
    print('something went wrong')
else:
    print('success')



## Write a DataFrame to a SQL database.

In [0]:
regions.write.format("jdbc").options(url="jdbc:mysql://localhost/northwind", driver='com.mysql.jdbc.Driver', dbtable='regions', user='test', password = "password", mode = "append", useSSL = "false").save()


## Read a SQL table into a Spark DataFrame.

In [0]:
regions2 = spark.read.format("jdbc").options(url="jdbc:mysql://localhost/northwind", driver="com.mysql.jdbc.Driver", dbtable= "regions", user="test", password="password").load()
regions2.show()


## Creating the regions2 DataFrame does not execute anything yet, but by making the DataFrame into a Temp View then running a Spark SQL query, it tells Spark to read the SQL data into a DataFrame and then use the cluster to do the processing, not the SQL source.

In [0]:
regions2.createOrReplaceTempView('regions2')
spark.sql('select * from regions2 where regionid < 3').show()

## Alternate ways to code a query using SQL and methods.

In [0]:
print(spark.sql('select count(*) from regions').collect())
spark.sql('select * from regions').count()

## Using SQL you can use familiar syntax instead of withColumn or withCoumnRenamed methods.

In [0]:
t1 = spark.sql('select TerritoryID as TerrID, UPPER(TerritoryName) as TerritoryName, RegionID from territories')
t1.show(5)

from pyspark.sql.functions import expr
territories.withColumn('TerritoryName', expr('UPPER(TerritoryName)')).withColumnRenamed('TerritoryID', 'TerrID').show(5)

## Sometimes there is a function in Python that doesn't exist in SQL and it would be helpful to use, so you could make a udf and use withColumn.

In [0]:
from pyspark.sql.functions import expr, udf
from pyspark.sql.types import *

t2 = spark.sql('select * from territories')
t2.printSchema()
#t2.show()
t2 = t2.withColumn('upperName', expr('UPPER(TerritoryName)'))
t2.show(5)

t2 = t2.withColumn('titleName', udf(lambda x : x.title(), StringType())(t2.upperName))
t2.show(5)


## To make it easier though, you could make the Python function into a udf that SQL can understand similar to how you can make a DataFrame seem like a virtual table with createOrReplaceTempView.

In [0]:
def reverseString(x):
    return x[::-1]

spark.udf.register('reverse', reverseString, StringType())

spark.sql('select *, reverse(TerritoryName) as Reversed from Territories').orderBy('Reversed').show()

## HQL has collect_set and collect_list functions to aggregate items into a list instead of summing them up. 

In [0]:
from pyspark.sql.functions import collect_list
territories.groupBy(territories.RegionID).agg(collect_list(territories.TerritoryName)).show()

tr1 = spark.sql("SELECT RegionID, collect_list(TerritoryName) AS TerritoryList FROM Territories GROUP BY RegionID")
tr1.show()
tr1.printSchema()
print(tr1.take(1))


## Instead of a simple datatype, you could also collect complex structured objects using the HQL NAMED_STRUCT.

In [0]:

sql = """SELECT r.RegionID, r.RegionName
, COLLECT_SET(NAMED_STRUCT("TerritoryID", TerritoryID, "TerritoryName", TerritoryName)) AS TerritoryList
FROM Regions AS r
JOIN Territories AS t ON r.RegionID = t.RegionID
GROUP BY r.RegionID, r.RegionName
ORDER BY r.RegionID"""

tr2 = spark.sql(sql)
tr2.printSchema()
print(tr2)
tr2.show()
print(tr2.take(2))
tr2.write.json('TerritoryRegion.json')
spark.sql('create table TerritoryRegion as ' + sql)

## If you have data that is already collected into a complex datatype and want to flatten it, you could use HQL EXPLODE function.

## You could use the Spark explode method.

In [0]:
from pyspark.sql.functions import explode
tr1.select('RegionID', explode('TerritoryList')).show()

## Or if the DataFrame is turned into a Temp View, you could use the HQL query to do it.

In [0]:
tr1.createOrReplaceTempView('RegionTerritories')
sql = """SELECT RegionID, TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS TerritoryName
ORDER BY RegionID, TerritoryName
"""
spark.sql(sql).show()

## Or you could select specific elements from a collection.

In [0]:
tr2.createOrReplaceTempView('RegionTerritories')
spark.sql("select RegionId, RegionName, TerritoryList[0] as First, TerritoryList[size(TerritoryList) - 1] as Last, size(TerritoryList) as TerritoryCount from RegionTerritories").show()

## If the array is of structs, note the syntax of fetching the elements from the struct uses the . like an object property.

In [0]:
sql = """SELECT RegionID, RegionName, Territory.TerritoryID AS TerritoryID
, Territory.TerritoryName AS TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS Territory
"""
spark.sql(sql).show()

## Homework ##

**First Challenge**

Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using Spark SQL as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

**Second Challenge**

Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.


# End of notebook